In [1]:
import numpy as np
import pandas as pd
import re
import math
import itertools
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import collections 

from ast import literal_eval
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import namedtuple
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
from  scipy import spatial
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

In [2]:
d2v_model = Doc2Vec.load('../model/word_embedding/Doc2vec_new_small2_4.model')

# modeling

### 데이터셋 불러오기

In [3]:
final_list  = pd.read_excel('./new_data/final_list_score4.xlsx')
inv_list  = pd.read_excel('./new_data/invest_df.xlsx')
final_list = final_list[final_list['established_date2'] < 6.0].reset_index(drop=True)
final_list.fillna('nan', inplace=True)
final_list['token'] = final_list.token.apply(lambda x:literal_eval(x))

In [4]:
final_list['invest'] = [str(x).split(',') for x in final_list['invest']]

In [5]:
invest = list(set([el.strip() for el in final_list['invest'].sum()]))
inv_list.set_index('invest', inplace=True)
inv_list = inv_list.apply(minmax_scale)

In [6]:
final_list.shape

(47, 11)

In [7]:
inv_list = inv_list.where(inv_list > np.quantile(inv_list['invest_num'], 0.25), np.quantile(inv_list['invest_num'], 0.25))

### Document vector

In [8]:
paragraph_id = {k:v for k, v in enumerate(d2v_model.docvecs.doctags.keys())}
doc_vec = {paragraph_id[k]:list(v) for k,v in enumerate(d2v_model.docvecs.vectors_docs)}

In [9]:
inv_tech_df = pd.DataFrame(columns = ['investor', 'tech', 'weights', 'company'])
for values_list in  final_list[['company', 'predicted_small_label1', 'predicted_small_label2', 'invest']].values:
#     print(values_list)
    for com in values_list[-1]:
#         print(com)
        if com.strip() in ['nan', '비공개', '크라우드펀딩', '사모펀드']:
            continue
        else:
            try:
                weights = inv_list.loc[com.strip()].values[0]
                inv_tech_df = inv_tech_df.append({'investor':com.strip(), 'tech':values_list[1], 'weights':weights, 'company':values_list[0]}, \
                                                 ignore_index=True)
            except KeyError:
                print(com)
                continue

옐로우독
 콜라보레이티브펀드
 알토스벤처스
옐로금융그룹
 KB인베스트먼트
 한화인베스트먼트
메가인베스트먼트
 스마일게이트인베스트먼트
 미래에셋벤처투자
현대기술투자
 비씨카드(BC카드)
 UTC인베스트먼트
사이버에이전트벤처즈 코리아
 PIA
 Mission Capital
 삼성벤처투자
 GC녹십자Genome
 스파크랩
 데일리파트너스
 원익
 Indiegogo
 동문파트너즈
피앤아이인베스트먼트
 카카오페이지
 신한금융투자
 디에스씨인베스트먼트
스트롱벤처스
 KT인베스트먼트
 미래에셋벤처투자
 스프링캠프
라인플러스
스프링캠프
SAP
 테크스타즈
 500 스타트업스
 신한금융투자
카이트창업가재단
 현대자동차 제로원엑셀러레이터
킹슬리 벤처스
 GS홈쇼핑
 DS앤파트너스
 뮤렉스파트너스
 신용보증기금 경기스타트업지점


In [10]:
inv = list(inv_list.index)
inv_idx = ["i%d" % i for i in range(1,len(inv_list.index)+1)]
inv_node = pd.DataFrame({'node':inv,'node_idx':inv_idx})

In [11]:
inv_tech_df.rename(columns={'investor':'node'}, inplace=True)

In [12]:
inv_tech_df.to_excel('./new_data/final_result/investor_graph2.xlsx', index=False, encoding='cp949')

### 네트워크

In [13]:
def network_score(G) :
    tt =  { "degree":dict(G.degree()), 
            "degree_centrality":nx.degree_centrality(G), #normalized degree, degree가 클수록 높음
            "closeness_centrality":nx.closeness_centrality(G), 
            "betweenness_centrality":nx.betweenness_centrality(G), 
            "pagerank":nx.pagerank(G)}
    tt_df = pd.DataFrame(tt)
    tt_df['node'] = list(tt_df.index)
    tt_df = pd.merge(tt_df,node_df,how='left')
    return tt_df

In [14]:
small_label = list(set(final_list.predicted_small_label1.tolist() + final_list.predicted_small_label2.tolist()))
comb_dv = list(itertools.combinations(small_label,2))

In [15]:
len(small_label)

33

In [16]:
%%time
tech_similarity_list = list()
tech_mat = pd.DataFrame(index=small_label, columns=small_label)
for comb1, comb2 in comb_dv:
    cos_sim = cosine_similarity([doc_vec[comb1]], [doc_vec[comb2]])[0][0]
    tech_mat.loc[comb1,comb2] = cos_sim
    tech_mat.loc[comb2,comb1] = cos_sim
    tech_similarity_list.append(cos_sim)
tech_mat2 = tech_mat.where(tech_mat > 0.13, 0.0)
tech_mat3 = tech_mat2.where(tech_mat2 == 0, 1)

Wall time: 203 ms


In [17]:
t_edge_df = pd.DataFrame(columns= ['source', 'target', 'weights'])
for idx, comb in enumerate(comb_dv):
    edge =tech_mat3.loc[comb[0], comb[1]]
    if edge == 0.0:
        continue
    else:
        weight = tech_mat2.loc[comb[0], comb[1]]
        t_edge_df = t_edge_df.append({'source':comb[0], 'target':comb[1], 'weights':weight}, ignore_index=True)
t_edge = [tuple(element) for element in t_edge_df.values]

In [18]:
%%time
company = list(final_list['company'])
company_idx = ["c%d" % i for i in range(1,len(company)+1)]
node_df = pd.DataFrame({'node':company,'node_idx':company_idx, 'token':final_list.token.tolist()})
sim_sentence = pd.DataFrame(columns=company, index=company)
comb_c = list(itertools.combinations(node_df.node.tolist(),2))
similarity_list = list()
for idx,comb in enumerate(comb_c):
    if idx % 100 == 0:
        print(comb[0], comb[1])
    sent1 = node_df.loc[node_df.node == comb[0], 'token'].values[0]
    sent2 = node_df.loc[node_df.node == comb[1], 'token'].values[0]
    sent1_vector = d2v_model.infer_vector(sent1)
    sent2_vector = d2v_model.infer_vector(sent2)
    sim_sentence.loc[comb[0], comb[1]] = cosine_similarity([sent1_vector], [sent2_vector])[0][0]
    sim_sentence.loc[comb[1], comb[0]] = cosine_similarity([sent2_vector], [sent1_vector])[0][0]
    similarity_list.append(cosine_similarity([sent1_vector], [sent2_vector])[0][0])
sim_sentence2 = sim_sentence.where(sim_sentence > np.quantile(similarity_list, 0.90), 0.0)
sim_sentence3 = sim_sentence2.where(sim_sentence2 == 0, 1)

렌딧 어니스트펀드
매스프레소 티엔디엔
마켓잇 스마트토이
제노플랜 아모랩
베이글랩스 레이지앤트
티엔디엔 스마트토이
스튜디오씨드코리아 밍글
플러스티브이 (주)블루웨일컴퍼니
메저차이나 알고리즘랩스
스마트토이 뷰티패스
스켈터랩스 (주)블루웨일컴퍼니
Wall time: 13.9 s


In [19]:
edge_df = pd.DataFrame(columns= ['source', 'target', 'weights'])
for idx,comb in enumerate(comb_c):
    edge =sim_sentence3.loc[comb[0], comb[1]]
    if edge == 0.0:
        continue
    else:
        weight = sim_sentence2.loc[comb[0], comb[1]]
        edge_df = edge_df.append({'source':comb[0], 'target':comb[1], 'weights':weight}, ignore_index=True)
c_edge = [tuple(element) for element in edge_df.values]

In [20]:
small = list(set(list(final_list['predicted_small_label1']) + list(final_list['predicted_small_label2'])))
small_idx = ["s%d" % i for i in range(1,len(small)+1)]
small_node = pd.DataFrame({'node':small,'node_idx':small_idx})
node_df = node_df.append(pd.DataFrame(data=small_node),ignore_index=True)

In [21]:
s_edge = []
for i in range(len(final_list)):
    company = final_list['company'][i]
    small1 = final_list['predicted_small_label1'][i]
    if small1 == 'nan':
        print(company)
    else:
        s_edge.append((small1, company,1))
    small2 = final_list['predicted_small_label2'][i]
    if small2 == 'nan':
        print(company)
    else:
        s_edge.append((small2, company,1))

In [22]:
company_tech = final_list[['company','predicted_label1','predicted_label2', 'predicted_small_label1', 'predicted_small_label2']]
company_tech.rename(columns={'company':'node'}, inplace=True)

C:\Users\lmhoon012\Anaconda3\envs\project\lib\site-packages\pandas\core\frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
node_df = node_df.append(pd.DataFrame(data=inv_node),ignore_index=True)
node_df.to_excel('./new_data/final_result/new_node_df2.xlsx', index=False, encoding='cp949')

In [24]:
i_edge = [(el[0], el[3],el[2]) for el in inv_tech_df.values]

### Graph(T-C-I)

In [25]:
nondirect_G = nx.Graph()
nondirect_G.add_weighted_edges_from(c_edge+s_edge+i_edge)
nondirect_score = network_score(nondirect_G)
nondirect_score.to_excel('./new_data/final_result/network_score3.xlsx', index=False, encoding='cp949')

#### NTBF

In [26]:
company_network_result = nondirect_score[nondirect_score['node_idx'].isin(company_idx)].sort_values(["pagerank"], ascending=[False])
company_network_result['total_weight'] = company_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
company_network_result = company_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
company_network_result = pd.merge(company_network_result, company_tech)
company_network_result.to_excel('./new_data/final_result/company_network_result_new2.xlsx', index=False, encoding='cp949')

In [27]:
com_score1 = company_network_result.groupby('predicted_small_label1').mean()[['total_weight']].sort_values('total_weight', ascending=False)

In [28]:
com_score2 = company_network_result.groupby('predicted_small_label2').mean()[['total_weight']].sort_values('total_weight', ascending=False)

In [29]:
com_score2.rename(columns ={'total_weight':'total_weight2'}, inplace=True)

In [30]:
total_index = list(set(com_score1.index.tolist() + com_score2.index.tolist()))

In [31]:
company_total_df = pd.DataFrame(index=total_index)

In [32]:
com_score1.index.name = None
com_score2.index.name = None

In [33]:
company_total_df = company_total_df.join(com_score1)
company_total_df = company_total_df.join(com_score2)

In [34]:
company_total_df.fillna(0, inplace=True)

In [35]:
company_score = pd.DataFrame(company_total_df.mean(axis=1), columns=['total_weight']).sort_values('total_weight', ascending=False)

In [36]:
company_score.to_excel('./new_data/final_result/company_based.xlsx')

#### Emerging Technology

In [37]:
tech_network_result = nondirect_score[nondirect_score['node_idx'].isin(small_idx)].sort_values(["pagerank"], ascending=[False])
tech_network_result['total_weight'] = tech_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
tech_network_result = tech_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
tech_network_result.to_excel('./new_data/new_experiment/tech_network_result_new2.xlsx', index=False, encoding='cp949')

In [38]:
tech_network_result[['node', 'total_weight']].sort_values('total_weight', ascending=False).to_excel('emerging_tech2.xlsx', index=False)

In [39]:
tech_score = tech_network_result[['node', 'total_weight']].sort_values('total_weight', ascending=False).set_index('node')

In [40]:
tech_score.index.name=None

#### investor

In [41]:
inv_network_result = nondirect_score[nondirect_score['node_idx'].isin(inv_idx)].sort_values(["pagerank"], ascending=[False])
inv_network_result['total_weight'] = inv_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
inv_network_result = inv_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
inv_network_result = pd.merge(inv_network_result, inv_tech_df)
inv_network_result.to_excel('./new_data/final_result/inv_network_result_new.xlsx', index=False, encoding='cp949')

In [42]:
inv_score = inv_network_result.groupby('tech').mean()[['total_weight']]

In [43]:
inv_score.sort_values('total_weight', ascending=False).to_excel('inv_tech2.xlsx')

In [44]:
inv_score.index.name=None

#### Total weight

In [52]:
total_weight_index = list(set(company_score.index.tolist() + inv_score.index.tolist() + tech_network_result.node.tolist()))
total_score = pd.DataFrame(index=total_weight_index)

In [53]:
company_score.rename(columns={'total_weight':'NS'}, inplace=True)
tech_score.rename(columns={'total_weight':'TS'}, inplace=True)
inv_score.rename(columns={'total_weight':'IS'}, inplace=True)

In [54]:
company_score.to_excel('./new_data/final_result/NS.xlsx')
tech_score.to_excel('./new_data/final_result/TS.xlsx')
inv_score.to_excel('./new_data/final_result/IS.xlsx')

In [55]:
total_score = total_score.join(company_score)
total_score = total_score.join(tech_score)
total_score = total_score.join(inv_score)

In [56]:
total_score_df = pd.DataFrame(total_score.fillna(0).sum(axis=1), columns=['Total score']).sort_values('Total score', ascending=False)

In [57]:
total_score_df.to_excel('./new_data/final_result/TOS.xlsx')

### 네트워크 그리키

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
edge = []
for i in range(len(final_list)):
    weight_sum = 0 
    for j in range(len(final_list['invest'][i])):
        if final_list['invest'][i][j] != 'nan':
            company = final_list['company'][i]
            invest = final_list['invest'][i][j]
            company = node_df[node_df['node']==company]['node_idx'].values[0]
            invest = node_df[node_df['node']==invest]['node_idx'].values[0]
            edge.append((company,invest))

In [ ]:
s_edge = []
for i in range(len(final_list)):
    company = final_list['company'][i]
    small1 = final_list['predicted_small_label1'][i]
    small2 = final_list['predicted_small_label2'][i]
    small1 = node_df[node_df['node']==small1]['node_idx'].values[0]
    small2 = node_df[node_df['node']==small2]['node_idx'].values[0]
    company = node_df[node_df['node']==company]['node_idx'].values[0]
    s_edge.append((small1,company))
    s_edge.append((small2,company))

In [ ]:
s_edge = pd.DataFrame(s_edge,columns=['source','target'])
edge = pd.DataFrame(edge,columns=['source','target'])
df_edges = s_edge.append(edge, ignore_index=True)

In [ ]:
df_nodes = pd.DataFrame()
df_nodes['name'] = node_df['node_idx']

In [ ]:
group = []
for i in range(len(df_nodes)):
    if df_nodes['name'][i][0] == 'c' :
        group.append(0)
    elif df_nodes['name'][i][0] == 's' :
        group.append(1)
    elif df_nodes['name'][i][0] == 'i' :
        group.append(2)     
        
df_nodes['group'] = group

In [ ]:
a = nondirect_score[['node_idx','pagerank']].copy()
a.columns = ['name','nodesize']
df_nodes = pd.merge(df_nodes,a,how = 'left')

In [ ]:
temp = []
for i in range(len(df_nodes)):
    if df_nodes['group'][i] == 0:
        temp.append(df_nodes['nodesize'][i]*10000)
    else :
        temp.append(20)

In [ ]:
df_nodes['nodesize'] = temp

In [ ]:
df_nodes['nodesize'] = df_nodes['nodesize']*5

In [ ]:
color_map = {0:'#ffdf55', 1:'#eebcbc', 2:'#72bbd0'}

In [ ]:
classes = np.array(df_nodes['group'])
classnames, indices = np.unique(classes, return_inverse=True)

In [ ]:
G = nx.Graph(day="Stackoverflow")

for index, row in df_nodes.iterrows():
    G.add_node(row['name'], group=row['group'], nodesize=row['nodesize'])
    
for index, row in df_edges.iterrows():
    G.add_edges_from([(row['source'], row['target'])])

In [ ]:
nx.draw()

In [ ]:
plt.figure(figsize=(25,25))
options = {
    'edge_color': '#FFDEA2',
    'width': 1,
    'with_labels': True,
    'font_weight': 'regular',
}
colors = [color_map[G.nodes[node]['group']] for node in G]
sizes = [G.nodes[node]['nodesize']*10 for node in G]

nx.draw(G, node_color=colors, node_size=sizes, pos=nx.spring_layout(G, k=0.25, iterations=50), **options)
ax = plt.gca()
ax.collections[0].set_edgecolor("#555555") 
plt.show()